In [ ]:
import pandas as pd
import numpy as np
from keras.layers import Input, Embedding, Flatten, Dot
from keras.models import Model
import redis
import fakeredis

In [ ]:
# create a fake Redis database
fake_redis = fakeredis.FakeStrictRedis()